# Leto Demo Notebook

# Setup

### Move to root of repo to run code. 

In [3]:
cd ..

/root/aEye


### Setup env using script

In [4]:
! bash reconstruction_setup.sh

Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://security.debian.org/debian-security buster/updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1).
libxext6 is already the newest version (2:1.3.3-1+b2).
ffmpeg is already the newest version (7:4.1.11-0+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 67 not upgraded.
Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda-client==1.7.2=py37_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::bokeh==1.4.0=py37_0
  - defaults/noarch::dask==2.11.0=py_0
  - defaults/linux-64::distributed==2.11.0=py37_0
  - defaults/linux-64::spyder==4.0.1=py37_0
  - defaults/linux-64::watchd

### Install all Requirements 

In [6]:
!pip install -r requirements_aEye.txt

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached opencv_python_headless-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.1 MB)
  Using cached opencv_contrib_python-4.8.0.74-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (67.8 MB)
  Using cached static_ffmpeg-2.5-py2.py3-none-any.whl (7.4 kB)
  Using cached mediapipe-0.9.0.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.0 MB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)


### Imports

In [7]:
import boto3
import cv2
import logging
import sys
from aEye.video import Video
from aEye.labeler import Labeler
from aEye.auxiliary import Aux
from aEye.RealBasicVSR_reconstruction_module.builder import download_model
root = logging.getLogger()
root.setLevel(logging.INFO)
root.setLevel(logging.DEBUG)
root.setLevel(logging.ERROR)

............................................................................................................................................................................................................................................................................................................................................................................................................................................
Download of https://github.com/zackees/ffmpeg_bins/raw/main/v5.0/linux.zip -> /opt/conda/lib/python3.7/site-packages/static_ffmpeg/bin/linux.zip completed.
Extracting /opt/conda/lib/python3.7/site-packages/static_ffmpeg/bin/linux.zip -> /opt/conda/lib/python3.7/site-packages/static_ffmpeg/bin


/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


### Create objects to process original video into the correct sizing

In [6]:
aux = Aux()

In [7]:
labeler = Labeler()

# Optional: Section 1: Trimming original video to 10 seconds. 

In [7]:
# video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [8]:
# #print the metadata
# print(video_list_s3[0].get_meta_data())

In [9]:
# trimmed_s3 = labeler.trim_video_start_end(video_list_s3,0,10)

In [10]:
# aux.execute_label_and_write_local(trimmed_s3)

In [11]:
# aux.upload_s3(trimmed_s3, bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [12]:
# aux.clean()

# Section 2: Downsampling trimmed video

### Grab the original Video from S3

In [6]:
video_list_s3 = aux.load_s3(bucket = 'leto-dish', prefix = 'original-videos/random-videos/')

In [7]:
print(video_list_s3)

[demo_10_second_clip.mp4]


In [8]:
#print the metadata
print(video_list_s3[0].extract_metadata())

{'streams': [{'index': 0, 'codec_name': 'h264', 'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10', 'profile': 'High', 'codec_type': 'video', 'codec_tag_string': 'avc1', 'codec_tag': '0x31637661', 'width': 1920, 'height': 1080, 'coded_width': 1920, 'coded_height': 1080, 'closed_captions': 0, 'film_grain': 0, 'has_b_frames': 2, 'pix_fmt': 'yuv420p', 'level': 40, 'color_range': 'tv', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'field_order': 'progressive', 'refs': 1, 'is_avc': 'true', 'nal_length_size': '4', 'id': '0x1', 'r_frame_rate': '25/1', 'avg_frame_rate': '25/1', 'time_base': '1/12800', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 128000, 'duration': '10.000000', 'bit_rate': '79042', 'bits_per_raw_sample': '8', 'nb_frames': '250', 'extradata_size': 50, 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0, 'visual_impair

From the metadata we can see that the original codec is h.264, and the video is 1080p

### Downsample the videos

In [9]:
downsampled_video = labeler.change_resolution(video_list_s3,"240p")

In [10]:
aux.execute_label_and_write_local(downsampled_video, "reduced_videos")

[resized_426x240_demo_10_second_clip.mp4]

### Save to S3

In [24]:
aux.upload_s3(downsampled_video, bucket = 'leto-dish', prefix = 'reduced-videos/')

# Section 3: Reconstruction Video

Here we will use the realbasicvsr module to super resolve the video 4x.

### Download the pretrained model

In [11]:
download_model("aEye/RealBasicVSR_reconstruction_module/RealBasicVSR_x4.pth","leto-dish","pretrained-models/RealBasicVSR_x4.pth")

Downloaded file to: aEye/RealBasicVSR_reconstruction_module/RealBasicVSR_x4.pth


Please provide some time for the model to be downloaded. It is a 200mb pretrained model. :)

### Run the reconstruction_realbasicvsr.py giving inputs and outputs

In [12]:
%%time

!python reconstruction_realbasicvsr.py /root/aEye/reduced_videos/resized_426x240_demo_10_second_clip.mp4 /root/aEye/reconstructed_videos/reconstructed_demo.mp4

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
2023-07-11 21:40:26,211 - mmedit - INFO - load checkpoint from torchvision path: torchvision://vgg19
load checkpoint f

### Save reconstructed video to S3

In [10]:
recon_vid_obj = aux.load_local("/root/aEye/reconstructed_videos/upsampled_demo.mp4")

In [12]:
aux.set_local_path("/root/aEye/reconstructed_videos/")

In [13]:
aux.upload_s3(recon_vid_obj, bucket = 'leto-dish', prefix = 'reconstructed-videos/')

In [14]:
aux.clean()